In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from collections import defaultdict

import random

from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.dummy import *

# 1.3 Regressors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.tree import DecisionTreeRegressor

from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.dummy import DummyRegressor
import seaborn as sns
import pickle
from scipy.stats import spearmanr
from sklearn.inspection import permutation_importance
import time
from sklearn.impute import SimpleImputer, KNNImputer

In [3]:
p = pd.read_csv('../data/proteomics_raw.csv', index_col=0, low_memory=False)
m = pd.read_csv('../data/mzmine/all_metabolites_unscaled.csv', index_col=0, low_memory=False)
df = pd.concat([p,m])
df = df.dropna(axis=1)
df.head()

,A1,A2,A3,A4,A5,A6,A8,A9,A10,A11,...,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12
1/sp|Q13151|ROA0_HUMAN,17.136855,16.981031,17.380600,17.313682,17.171079,16.907761,16.866405,17.002087,16.914275,17.070147,...,16.278393,16.249226,16.234352,16.511556,16.769266,16.354346,16.125869,16.294465,16.611128,16.244718
1/sp|P25705|ATPA_HUMAN,19.765949,19.787863,19.945158,19.935565,19.979122,19.673296,19.869383,19.764727,19.675089,19.819422,...,19.055441,19.114331,19.125909,19.341502,19.518779,19.176859,18.821754,18.812864,19.208747,18.838438
1/sp|P17987|TCPA_HUMAN,19.558667,19.797518,19.878208,19.990131,20.262627,19.490869,19.623488,19.520840,19.593644,19.742180,...,19.301966,19.440957,19.142047,19.409105,19.936231,19.311178,19.078091,18.963039,19.533181,19.491008
1/sp|P06576|ATPB_HUMAN,19.792755,20.061846,20.220668,20.048093,20.013875,19.968291,19.903663,19.948603,19.918656,20.073732,...,18.933254,19.264677,19.163665,19.404896,19.397101,19.329793,19.010495,19.187193,19.580935,19.216481
1/sp|P62913|RL11_HUMAN,17.572020,16.776782,17.838812,16.827748,17.922630,16.808190,17.253954,17.459773,17.828790,16.989804,...,16.741861,16.951118,16.889754,17.053418,16.492265,16.294084,15.721132,16.464488,16.669949,17.131954


In [ ]:
ss = StandardScaler()
df_ss = ss.fit_transform(df.T )

In [17]:
df_ss

array([[ 0.89632589,  0.72015407,  0.36835027, ...,  2.02550768,
        -1.31610904,  1.9146289 ],
       [ 0.61702833,  0.76436164,  0.84799835, ...,  1.46278463,
         0.04064   ,  0.96667264],
       [ 1.33321161,  1.08168646,  1.01003677, ...,  2.13153292,
        -1.61016303,  1.66196422],
       ...,
       [-0.61356805, -1.20258703, -0.82775938, ..., -1.41029547,
         0.14779885, -0.51633102],
       [-0.04598231, -0.40393811,  0.31717055, ...,  0.17698047,
        -0.780326  , -0.85857095],
       [-0.70273265, -1.15099456,  0.232481  , ..., -0.99185815,
        -0.62529476,  0.29176898]])

In [18]:
df_ss = pd.DataFrame(df_ss, columns= df.index, index=df.columns)

In [ ]:
plt.hist(df_ss.iloc[4,:])

In [ ]:
plt.hist(df_ss.iloc[:,4])

In [9]:
np.mean(df_ss.iloc[:,4])

1.403822857417797e-14

In [10]:
np.std(df_ss.iloc[:,4])

0.9999999999999999

In [ ]:
p = df_ss.iloc[:,:450]
m = df_ss.iloc[:,450:]
m

# make full X data (proteins with KNN imputation)

# Split train/test

In [ ]:
### get one random index from each letter
test_index = []
group_letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
for letter in group_letters:
    print(letter)    
    test_index.append(random.choice(p[[letter in s for s in p.index]].index))

In [14]:
test_index = ['A12', 'B9', 'C11', 'D3', 'E9', 'F5', 'G2']

In [ ]:
p_test = p.loc[test_index]
m_test = m.loc[test_index]
m_test

In [ ]:
p_train = p.drop(index=test_index)
m_train = m.drop(index=test_index)
m_train

In [ ]:
p_train.iloc[:,1].hist()

# START HERE to Load saved data

In [71]:
import seaborn as sns

In [72]:
def model_eval(model, model_name, XTRAIN, XTEST,YTRAIN, YTEST):
    model.fit(XTRAIN, YTRAIN)
    YPRED = model.predict(XTEST)
    print('Mean squared error: %.4f'
      % mean_squared_error(YTEST, YPRED))
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.4f'
      % r2_score(YTEST, YPRED))
    # Plot outputs
    plt.rcParams['figure.figsize'] = 5,5
    #sns.jointplot(YTEST, YPRED,  color='black', kind='reg')
    plt.scatter(YTEST, YPRED, color='blue')
    plt.xlabel('true')
    plt.ylabel('pred')
    plt.subplots_adjust(top=0.9)
    plt.suptitle(model_name, fontsize = 16)
    plt.show()
    return mean_squared_error(YTEST, YPRED), r2_score(YTEST, YPRED)

In [73]:
## try many models
from sklearn.ensemble import ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor


# Hyperparameter search

In [147]:
md = np.arange(10,90,20)
md = np.append(md, None)
search_params = {
        'max_depth': md,
        'min_samples_leaf': [1,2,5],
        'min_samples_split':[2,5,10],
        'max_features': ['log2', 'auto', 'sqrt'],
        'n_estimators': [500, 1000, 1500]
    }

In [75]:
from sklearn.model_selection import ParameterGrid

In [ ]:
# grid search - depth and estimators -- OOPS that was the wrong way - protein to metabolite. 
etgs_p_to_m = RandomizedSearchCV(
    estimator=ExtraTreesRegressor(random_state = 42),
    param_distributions=search_params,
    cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1, n_iter=100)
grid_result_met = etgs_p_to_m.fit(p_train, m_train)

In [ ]:
et_best_met = grid_result_met.best_estimator_
et_best_met

In [ ]:
et_best_met.get_params()

In [151]:
ypred = et_best_met.predict(p_test)

In [ ]:
p_test

In [ ]:

print('Mean squared error: %.4f'
  % mean_squared_error(m_test, ypred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.4f'
  % r2_score(np.asarray(m_test).flatten(), ypred.flatten()))
# Plot outputs
plt.rcParams['figure.figsize'] = 5,5
#sns.jointplot(YTEST, YPRED,  color='black', kind='reg')
ax = sns.jointplot(np.asarray(m_test).flatten(), ypred.flatten(), kind='reg')
plt.xlabel('true')
plt.ylabel('pred')
plt.subplots_adjust(top=0.9)
plt.suptitle('extra trees', fontsize = 16)
ax.ax_marg_x.set_xlim(-6, 6)
ax.ax_marg_y.set_ylim(-6, 6)
plt.show()

In [154]:
dummy = DummyRegressor()
dummy.fit(p_train, m_train)
ypred_dummy = dummy.predict(p_test)

In [ ]:
print('Mean squared error: %.4f'
  % mean_squared_error(m_test, ypred_dummy))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.4f'
  % r2_score(np.asarray(m_test).flatten(), ypred_dummy.flatten()))
# Plot outputs
plt.rcParams['figure.figsize'] = 5,5
#sns.jointplot(YTEST, YPRED,  color='black', kind='reg')
ax = sns.jointplot(np.asarray(m_test).flatten(), ypred_dummy.flatten(), kind='reg')
plt.xlabel('true')
plt.ylabel('pred')
plt.subplots_adjust(top=0.9)
plt.suptitle('dummy', fontsize = 16)
ax.ax_marg_x.set_xlim(-6, 6)
ax.ax_marg_y.set_ylim(-6, 6)
plt.show()

In [ ]:
# grid search - depth and estimators -- OOPS that was the wrong way - protein to metabolite. 
etgs_m_to_p = RandomizedSearchCV(
    estimator=ExtraTreesRegressor(random_state = 42),
    param_distributions=search_params,
    cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1, n_iter=100)
grid_result_m_to_p = etgs_m_to_p.fit(m_train, p_train)

In [157]:
et_best_m_to_p = grid_result_m_to_p.best_estimator_
et_best_m_to_p

ExtraTreesRegressor(max_depth=70, n_estimators=1500, random_state=42)

In [ ]:
et_best_m_to_p.get_params()

In [ ]:
ypred = et_best_m_to_p.predict(m_test)

In [160]:
ypred.shape

(7, 450)

In [161]:
p_test.shape

(7, 450)

In [ ]:
plt.scatter(p_test.iloc[0,:], ypred[0,:])

In [163]:
r2_score(p_test.iloc[0,:], ypred[0,:])

0.13908445678859682

In [164]:
r2_score(np.asarray(p_test), ypred)

0.6252320956013405

In [ ]:

print('Mean squared error: %.4f'
  % mean_squared_error(ypred, p_test))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.4f'
  % r2_score(np.asarray(p_test).flatten(), ypred.flatten()))
# Plot outputs
plt.rcParams['figure.figsize'] = 5,5
#sns.jointplot(YTEST, YPRED,  color='black', kind='reg')
ax = sns.jointplot(np.asarray(p_test).flatten(), ypred.flatten(), kind='reg')

plt.subplots_adjust(top=0.9)
plt.suptitle('extra trees', fontsize = 16)
plt.xlabel('true')
plt.ylabel('pred')
ax.ax_marg_x.set_xlim(-3, 3)
ax.ax_marg_y.set_ylim(-3, 3)
plt.show()

In [166]:
dummy = DummyRegressor()
dummy.fit(m_train, p_train)
ypred_dummy = dummy.predict(m_test)


In [ ]:
print('Mean squared error: %.4f'
  % mean_squared_error(p_test, ypred_dummy))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.4f'
  % r2_score(np.asarray(p_test).flatten(), ypred_dummy.flatten()))
# Plot outputs
plt.rcParams['figure.figsize'] = 5,5
#sns.jointplot(YTEST, YPRED,  color='black', kind='reg')
ax = sns.jointplot(np.asarray(p_test).flatten(), ypred_dummy.flatten(), kind='reg')
plt.xlabel('true')
plt.ylabel('pred')
plt.subplots_adjust(top=0.9)
plt.suptitle('dummy', fontsize = 16)
ax.ax_marg_x.set_xlim(-3, 3)
ax.ax_marg_y.set_ylim(-3, 3)
plt.show()

In [88]:
#from joblib import dump, load

filename = 'best_et.pickle'
with open(filename, 'wb') as fh:
    pickle.dump(et_best_met, fh)


In [ ]:
with open('best_et.pickle', 'rb') as fh:
    et_best_met = pickle.load(fh)